# Flujo de trabajo ETL Reviews

Este notebook ejecuta un proceso ETL (Extract, Transform, Load) completo, desde la extracción de datos de una base de datos, la transformación de estos datos, hasta la carga de los datos transformados en un archivo CSV

## Lectura de los archivos descargados de la página de airbnb

Analizaremos los datos de la ciudad de Paris generados hasta el 4 de septiembre de 2023.

http://insideairbnb.com/get-the-data/

### Reviews
Archivo de opiniones de los usuarios de los alojamientos de airbnb.

In [1]:
# libraries
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup # Para eliminar html de description
from langdetect import detect, DetectorFactory # para detectar idiomas
from dask.dataframe import from_pandas
from dask.diagnostics import ProgressBar
import swifter
import fasttext

In [2]:
reviews_detailed = pd.read_csv('../data/reviews.csv.gz')
reviews_detailed.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3109,123127969,2016-12-27,12389804,Sophie,The host canceled this reservation the day bef...
1,3109,123274144,2016-12-28,67553494,Tom'S,The host canceled this reservation 2 days befo...
2,3109,207127433,2017-10-28,51636494,Patricia,Tout s'est bien déroulé. Merci bien. PG
3,3109,208779822,2017-11-03,4142888,Patricia,Un petit nid fouiller douillet situé dans app...
4,3109,234257457,2018-02-12,3539452,Dominique,The host canceled this reservation 8 days befo...


In [3]:
reviews_detailed.shape

(1619702, 6)

In [4]:
reviews_detailed.isnull().sum()

listing_id         0
id                 0
date               0
reviewer_id        0
reviewer_name      1
comments         102
dtype: int64

## Transformaciones

Para este caso hay 102 filas con el campo 'comments' vacío, para el análisis utilizaremos solo registros con reseñas del hospedaje por lo que eliminaremos las filas con 'comments' vacíos

In [5]:
reviews_detailed = reviews_detailed.dropna(subset=['comments'])

Convertir el Campo date a Fecha

In [6]:
reviews_detailed['date'] = pd.to_datetime(reviews_detailed['date'])

Eliminar los Campos reviewer_id y reviewer_name

In [7]:
reviews_detailed = reviews_detailed.drop(['reviewer_id', 'reviewer_name'], axis=1)

Eliminar HTML de la Columna comments con BeautifulSoup

In [8]:
reviews_detailed['comments'] = reviews_detailed['comments'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text() if pd.notna(x) else x)

/var/folders/7r/4sszslkn68z61lfg2zps83wm0000gp/T/ipykernel_13927/3172620809.py:1: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  reviews_detailed['comments'] = reviews_detailed['comments'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text() if pd.notna(x) else x)


In [9]:
reviews_detailed.isnull().sum()

listing_id    0
id            0
date          0
comments      0
dtype: int64

In [10]:
reviews_detailed.head()

,listing_id,id,date,comments
0,3109,123127969,2016-12-27,The host canceled this reservation the day bef...
1,3109,123274144,2016-12-28,The host canceled this reservation 2 days befo...
2,3109,207127433,2017-10-28,Tout s'est bien déroulé. Merci bien. PG
3,3109,208779822,2017-11-03,Un petit nid fouiller douillet situé dans app...
4,3109,234257457,2018-02-12,The host canceled this reservation 8 days befo...


## Asignación de idioma del comentario

Este conjunto de datos contiene reseñas de hospedajes en airbnb de la ciudad de París, estos están en diferentes idiomas. Haciendo uso de la librería **langdetect** vamos a asignar el idioma a cada comentario


### 1 intento
Usando langdetect, creando un dataframe de dask con 2 particiones para que funcione bien con mi procesador de 2 núcleos, pero para las capacidades de mi máquina el proceso tarda 17 horas, así que se abortó el proceso

In [11]:
DetectorFactory.seed = 0 #Semilla aleatoriedad

In [26]:
def langue_detect(text):
    try:
        # Verificar si el texto tiene una longitud mínima de 10 caracteres
        if len(text) > 10:
            return detect(text)
        else:
            return "unknown"
    except Exception as e:
        print(e)
        return "unknown"

In [30]:
# Convertir DataFrame de Pandas a un DataFrame de Dask
ddf_reviews = from_pandas(reviews_detailed, npartitions=2)

In [31]:
# Aplicar la función de detección de idiomas usando map_partitions
ddf_reviews['language'] = ddf_reviews['comments'].map_partitions(
    lambda df: df.apply(lambda x: langue_detect(x) if pd.notna(x) else "unknown"), meta='str'
)

/Users/diana/anaconda3/envs/big_data_project/lib/python3.11/site-packages/dask/dataframe/core.py:7246: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


In [32]:
# Calcular y persistir los resultados
with ProgressBar():
    ddf_reviews = ddf_reviews.persist()

[                                        ] | 0% Completed | 1.62 ms

[                                        ] | 0% Completed | 7.25 s msNo features in text.
[                                        ] | 0% Completed | 129.30 sNo features in text.
[                                        ] | 0% Completed | 10m 52ss


KeyboardInterrupt: 

In [35]:
reviews_detailed['language'] = reviews_detailed['comments'].swifter.apply(langue_detect)

Pandas Apply:   0%|          | 0/1619600 [00:00<?, ?it/s]

No features in text.
No features in text.


KeyboardInterrupt: 

### 2 segundo intento
Con fasttext y swifter se logró agregar la columna para el lenguaje del comentario descargando previamente el archivo de entrenamiento de fasttext lid.176.ftz

In [38]:
# Cargar el modelo preentrenado de detección de idioma
model = fasttext.load_model('../lid.176.ftz')

In [39]:
# Función para detectar idioma utilizando fastText
def detect_language_fasttext(text):
    if pd.isna(text) or text.strip() == '':
        return "unknown"
    else:
        # FastText espera una lista de textos para la predicción y devuelve una lista de tuplas
        predictions = model.predict([text])
        # El primer elemento de la tupla es el código de idioma con el prefijo '__label__'
        language = predictions[0][0][0].replace('__label__', '')
        return language

In [40]:
reviews_detailed['language'] = reviews_detailed['comments'].swifter.apply(detect_language_fasttext)

Pandas Apply:   0%|          | 0/1619600 [00:00<?, ?it/s]

In [41]:
reviews_detailed.isnull().sum()

listing_id    0
id            0
date          0
comments      0
language      0
dtype: int64

In [42]:
reviews_detailed.head()

,listing_id,id,date,comments,language
0,3109,123127969,2016-12-27,The host canceled this reservation the day bef...,en
1,3109,123274144,2016-12-28,The host canceled this reservation 2 days befo...,en
2,3109,207127433,2017-10-28,Tout s'est bien déroulé. Merci bien. PG,fr
3,3109,208779822,2017-11-03,Un petit nid fouiller douillet situé dans app...,fr
4,3109,234257457,2018-02-12,The host canceled this reservation 8 days befo...,en


No features in text.
No features in text.
No features in text.
No features in text.
No features in text.
No features in text.


In [43]:
#Escribir archivo csv
reviews_detailed.to_csv('../data/reviews_summary.csv', index=False)

No features in text.
No features in text.
No features in text.
No features in text.


KeyboardInterrupt: 

No features in text.
No features in text.


### Se escribió satisfactoriamente el archivo csv listo para subir a AWS :)

In [2]:
reviews = pd.read_csv('../data/reviews_summary.csv')


In [5]:
reviews.shape

(1619600, 5)

In [3]:
reviews.head()

,listing_id,id,date,comments,language
0,3109,123127969,2016-12-27,The host canceled this reservation the day bef...,en
1,3109,123274144,2016-12-28,The host canceled this reservation 2 days befo...,en
2,3109,207127433,2017-10-28,Tout s'est bien déroulé. Merci bien. PG,fr
3,3109,208779822,2017-11-03,Un petit nid fouiller douillet situé dans app...,fr
4,3109,234257457,2018-02-12,The host canceled this reservation 8 days befo...,en
